In [1]:
import pandas as pd
import numpy as np

## My F-beta score

Difference with default F1 score:
When deal with tricky metric like this
<pre>
|         |      |     True    |
|         |      |   0  |   1  |
|---------|------|------|------|
| Predict |   0  |   3  |   0  |
|         |   1  |   4  |   0  |
</pre>
No real positive samples makes the denominator and numeratore of F1 metric equal to 0. F1, recall, precision = 0/0.
Default F1 test are all equal to 0. But in my opinion: F1, recall, precision score are 0, 1, 0 respectively.

-------
And this:
          True
           0
Predict   0   3
In this case, I will set all of F1, recall, precision to 1. However, the default is 0.

--------
Besides, my version could return the cross table as above.

--------

Note about Beta:
![title](f1score.svg)
beta < 1 lends more weight to precision, while beta > 1 favors recall

In [2]:
def Score (PredictList,TrueList,beta):
    X=pd.concat([pd.DataFrame(PredictList),pd.DataFrame(TrueList)],axis=1, ignore_index=True)
    X['count']=1
    X.columns=['Predict','True','count']
    Acc=X.pivot_table(index='Predict',columns='True',values='count',aggfunc=np.sum)
    Acc=Acc.fillna(value=0)
    rate,f1,recall,precision=0,0,0,0
    b=beta**2
    if Acc.shape==(2,2):
        rate=rate+(float(Acc[0][0]+Acc[1][1])/Acc.sum().sum())
        if Acc[1][1]!=0:
            recall=float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])
            precision=float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])
            f1=(1+b) * (float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) * (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])) / ((b * float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) + (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])))
    elif Acc.shape==(1,2):
        if Acc.index[0]==0:
            rate=rate+(float(Acc[0][0])/Acc.sum().sum())
        else:
            rate=float(Acc[0][1])/Acc.sum().sum()
            recall=1
            precision=float(Acc[0][1])/Acc.sum().sum()
            f1=(1+b) * ((float(Acc[0][1])/Acc.sum().sum()) * 1) / ((b * float(Acc[0][1])/Acc.sum().sum()) + 1)
    elif Acc.shape==(2,1):
        if Acc.columns==1:
            rate=float(Acc.iloc[1])/Acc.sum().sum()
            recall=float(Acc.iloc[1])/Acc.sum().sum()
            precision=1
            f1=(1+b) * float(Acc.iloc[1])/Acc.sum().sum()/(float(Acc.iloc[1])/Acc.sum().sum()+ b)
        else:
            rate=float(Acc.iloc[0])/Acc.sum().sum()
            recall=1
    else: #Acc.shape=(1,1)
        rate=1
        recall=1
        precision=1
        f1=1
    return Acc,rate,f1,recall,precision

# Import Y

In [3]:
Y=pd.read_csv('comm-at-least-4-Y.csv',header=0,index_col=0)
Y=Y.sort_index(axis=0, level=None, ascending=True)
Y=Y.fillna(value=0)

In [8]:
Y=Y[Y.index.isin(new_metric.index)]

In [5]:
Y['Local'].sum()

109L

In [8]:
len(Y)

2692


# Import and Transform X

In [18]:
#commall=pd.read_csv("./commall.csv",header=None, names=['time','pageid','uid'],sep=',')
likeall=pd.read_csv("./like-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
likeall['count']=1

In [20]:
del likeall

In [5]:
commall=pd.read_csv("./comm-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
commall['count']=1

In [19]:
new_metric=likeall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [16]:
new_metric2=commall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [27]:
#Save metric
new_metric.to_csv(path_or_buf='./comm-at-least-4-engage-metric.csv')

In [21]:
new_metric=new_metric.fillna(value=0)

In [17]:
new_metric2=new_metric2.fillna(value=0)

In [18]:
new_metric_combined=pd.concat([new_metric,new_metric2],axis=1)

In [19]:
new_metric_combined=new_metric_combined.fillna(value=0)

In [14]:
new_metric[new_metric>0]=1

In [17]:
co_matrix=new_metric.dot(new_metric.T)

In [25]:
new_metric=np.asarray(new_metric,dtype="float64")

In [30]:
Y=np.asarray(Y,dtype="float64")

# Split Data

In [39]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_metric, Y, test_size=0.2)
y_train=np.asarray(y_train,dtype="float64")
y_test=np.asarray(y_test,dtype="float64")

# KNN

In [44]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train[:,1]) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [45]:
Acc,rate,f1,recall,precision=Score(neigh.predict(X_test),y_test[:,1],1)

In [49]:
print Acc
print rate
print f1
print recall
print precision

True     0.0  1.0
Predict          
0.0      512   23
1.0        2    2
0.953617810761
0.137931034483
0.08
0.5


# K-means

In [59]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(X_train)

ImportError: Building module sklearn.cluster._k_means failed: ["DistutilsFileError: could not create 'C:\\Program Files;C:': \xa4\xe5\xa5\xf3\xa6W\xa1B\xa5\xd8?\xa6W\xa9\xce\xa8\xf7??\xaak\xa4\xa3\xa5\xbf\xda\xcc\xa1C\n"]